# Preprocessing for CICFlowMeter flows

In [1]:
from pathlib import Path
import polars as pl
from ids_research.preprocessors import CicFlowMeterPreprocessor

## CIC-IDS-2017

In [2]:
cic_ids_2017_path = Path("~/data/CIC-IDS-2017_GeneratedLabelledFlows.parquet")
cic_ids_2017 = pl.read_parquet(cic_ids_2017_path.expanduser())
cic_ids_2017.head()

Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,…,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
str,str,i64,str,i64,i64,datetime[μs],i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,…,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""192.168.10.25-23.194.182.12-59…","""192.168.10.25""",59385,"""23.194.182.12""",443,6,2017-07-03 01:00:01,83823,10,5,713.0,479.0,469.0,0.0,71.3,147.287361,341.0,0.0,95.8,149.52993,14220.440691,178.948499,5987.357143,9181.991385,23509.0,3.0,83823.0,9313.666667,11242.816384,23994.0,3.0,51900.0,12975.0,12669.114649,24005.0,540.0,0,…,139.666269,19506.666667,0,0,0,1,0,0,0,0,0.0,79.466667,71.3,95.8,0,0,0,0,0,0,10,713,5,479,65535,972,5,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""BENIGN"""
"""192.168.10.25-23.194.182.12-59…","""23.194.182.12""",443,"""192.168.10.25""",59385,6,2017-07-03 01:00:01,3,2,0,37.0,0.0,37.0,0.0,18.5,26.162951,0.0,0.0,0.0,0.0,1.2333e7,666666.666667,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1,…,21.36196,456.333333,0,1,0,0,1,0,0,0,0.0,37.0,18.5,0.0,0,0,0,0,0,0,2,37,0,0,972,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""BENIGN"""
"""8.0.6.4-8.6.0.1-0-0-0""","""8.6.0.1""",0,"""8.0.6.4""",0,0,2017-07-03 01:00:01,118699862,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64027,1.5827e6,3.4595e6,1.6832267e7,0.0,1.18699862e8,1.5827e6,3.4595e6,1.6832267e7,0.0,0.0,0.0,0.0,0.0,0.0,0,…,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,76,0,0,0,-1,-1,0,0,3.751428e6,2.9310e6,6.49062e6,1.0,1.0124e7,4.0564e6,1.6832267e7,6.003507e6,"""BENIGN"""
"""192.168.10.1-192.168.10.3-53-6…","""192.168.10.3""",61098,"""192.168.10.1""",53,17,2017-07-03 01:00:01,86739,1,1,51.0,67.0,51.0,51.0,51.0,0.0,67.0,67.0,67.0,0.0,1360.403048,23.057679,86739.0,0.0,86739.0,86739.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,…,9.237604,85.333333,0,0,0,0,0,0,0,0,1.0,84.5,51.0,67.0,0,0,0,0,0,0,1,51,1,67,-1,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""BENIGN"""
"""192.168.10.3-192.168.10.25-53-…","""192.168.10.25""",51147,"""192.168.10.3""",53,17,2017-07-03 01:00:01,100934792,4,4,142.0,684.0,37.0,34.0,35.5,1.732051,211.0,131.0,171.0,46.188022,8.183501,0.079259,1.4419e7,3.8111e7,1.00847374e8,3.0,1.00934563e8,3.3645e7,5.8275e7,1.00934557e8,3.0,1.00847609e8,3.3616e7,5.8224e7,1.00847603e8,3.0,0,…,76.673732,5878.861111,0,0,0,0,0,0,0,0,1.0,107.875,35.5,171.0,0,0,0,0,0,0,4,142,4,684,-1,-1,3,32,87186.0,0.0,87186.0,87186.0,1.00847374e8,0.0,1.00847374e8,1.00847374e8,"""BENIGN"""


In [3]:
target_col = "Label"

In [4]:
cic_ids_2017.select(target_col).unique()

Label
str
"""Bot"""
"""Web Attack  Sql Injection"""
"""Web Attack  XSS"""
"""DoS GoldenEye"""
"""FTP-Patator"""
…
"""Infiltration"""
"""DoS slowloris"""
"""PortScan"""


In [5]:
preprocessor = CicFlowMeterPreprocessor()

In [6]:
preprocessor.fit(cic_ids_2017)

In [7]:
cic_ids_2017_transformed = preprocessor.transform(cic_ids_2017)
cic_ids_2017_transformed.head()

Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,…,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_17,Protocol_6,Label
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u8,u8,u8,bool
0.000699,0.000041,0.000017,0.000055,7.3079e-7,0.018896,0.0,0.012002,0.02067,0.01746,0.0,0.016516,0.018247,0.111927,0.333363,0.00005,0.000108,0.000196,1.4167e-7,0.000699,0.000078,0.000133,0.0002,1.2500e-7,0.0004325,0.000108,0.00015,0.0002,0.0000045,0.0,0.0,0.0,0.0,0.999856,0.994592,0.00004,0.00003,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.020411,0.012002,0.016516,0.0,0.0,0.0,0.0,0.0,0.0,0.000041,0.000055,0.000017,7.3079e-7,1.0,0.014847,0.000023,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,false
1.3333e-7,0.000005,0.0,0.000003,0.0,0.001491,0.0,0.003114,0.003672,0.0,0.0,0.0,0.0,0.11721,0.444444,1.3333e-7,0.0,1.3333e-7,1.4167e-7,2.5000e-8,2.5000e-8,0.0,2.5000e-8,1.2500e-7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.999856,0.994592,0.222222,0.0,…,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.009503,0.003114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000003,0.0,0.0,0.014847,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,false
0.989166,0.000341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111921,0.333333,0.013189,0.040795,0.140269,1.1667e-7,0.989166,0.013189,0.040891,0.140269,1.0000e-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,2.1342e-7,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000341,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.034104,0.039501,0.059006,9.0909e-9,0.084367,0.052749,0.140269,0.050029,1,0,0,false
0.000723,0.0,0.000003,0.000004,1.0222e-7,0.002055,0.021935,0.008585,0.0,0.003431,0.023135,0.011551,0.0,0.111922,0.333337,0.000723,0.0,0.000723,0.000723,0.0,0.0,0.0,0.0,1.0000e-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,0.000004,0.000006,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00641,0.021704,0.008585,0.011551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.000003,1.0222e-7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,false
0.841123,0.000014,0.000014,0.000011,0.000001,0.001491,0.014624,0.005976,0.000243,0.010804,0.045235,0.02948,0.005636,0.111921,0.333333,0.120161,0.449424,0.840395,1.4167e-7,0.841121,0.280374,0.688801,0.841121,1.2500e-7,0.840397,0.280132,0.689715,0.840397,2.5000e-8,0.0,0.0,0.0,0.0,0.999856,0.994592,1.3210e-8,1.9815e-8,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00641,0.027708,0.005976,0.02948,0.0,0.0,0.0,0.0,0.0,0.0,0.000014,0.000011,0.000014,0.000001,0.0,0.0,0.000014,1.0,0.0007926,0.0,0.0007926,0.0007926,0.840395,0.0,0.840395,0.840395,0,1,0,false


In [8]:
# constant columns
cic_ids_2017_transformed.select([
    pl.col(col).n_unique() for col in cic_ids_2017_transformed.columns
    if cic_ids_2017_transformed.select(pl.col(col).n_unique()).item() < 2
])


Bwd PSH Flags,Bwd URG Flags,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg
u32,u32,u32,u32,u32,u32,u32,u32
1,1,1,1,1,1,1,1


## MAWILab

In [9]:
mawilab_sample_2011_n10_000_path = Path("~/data/cicflowmeter_sample_2011-01-01_n10_000.parquet")
mawilab_sample_2011_n10_000 = pl.read_parquet(mawilab_sample_2011_n10_000_path.expanduser())
mawilab_sample_2011_n10_000.head()

Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,…,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,filename,Anomaly ID,Taxonomy,Heuristic,Detectors,Distance
str,str,i64,str,i64,i64,datetime[μs],i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,…,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,u16,str,f64,u8,f64
"""71.227.62.172-208.162.239.49-1…","""71.227.62.172""",1131,"""208.162.239.49""",445,6,2011-01-01 05:00:00,2949823,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678007,2.949823e6,0.0,2.949823e6,2.949823e6,2.949823e6,2.949823e6,0.0,2.949823e6,2.949823e6,0.0,0.0,0.0,0.0,0.0,0,…,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,65535,0,0,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""benign""","""data/cicflowmeter/v1.1/year=20…",null,null,null,null,null
"""208.31.103.165-19.220.88.113-4…","""208.31.103.165""",49522,"""19.220.88.113""",8908,6,2011-01-01 05:00:00,112726382,8,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.221776,4.6969e6,4.1383e6,1.4521369e7,987.0,1.07452006e8,1.5350e7,1.3002e7,3.9035203e7,2.178362e6,1.12612328e8,7.0383e6,6.0332e6,2.0807137e7,987.0,0,…,25,0,0,0,2.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,65178,65392,0,20,4945919.4,1.9706e6,7.56105e6,2.871024e6,9.185495e6,2.8411e6,1.4521369e7,5.47014e6,"""suspicious""","""data/cicflowmeter/v1.1/year=20…",68,"""alphflHTTP""",-0.12005,3,null
"""209.114.168.96-141.10.194.33-5…","""209.114.168.96""",51514,"""141.10.194.33""",48698,17,2011-01-01 05:00:00,65496520,520,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.939353,126197.533719,318157.256111,1.477244e6,1.0,6.549652e7,126197.533719,318157.256111,1.477244e6,1.0,0.0,0.0,0.0,0.0,0.0,0,…,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,520,0,0,0,0,0,0,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""anomalous""","""data/cicflowmeter/v1.1/year=20…",0,"""mptmp""",0.23749,7,null
"""168.203.32.74-18.41.105.157-80…","""168.203.32.74""",80,"""18.41.105.157""",1141,6,2011-01-01 05:00:00,5982238,39,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.519299,157427.315789,367188.848647,1.73255e6,1.0,5.982238e6,157427.315789,367188.848647,1.73255e6,1.0,0.0,0.0,0.0,0.0,0.0,0,…,39,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,19,0,0,0,108,0,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""anomalous""","""data/cicflowmeter/v1.1/year=20…",9,"""mptmp""",0.13257,7,null
"""209.114.168.106-97.241.163.189…","""209.114.168.106""",3125,"""97.241.163.189""",49347,6,2011-01-01 05:00:00,117767159,34,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.551937,1.8401e6,2.5322e6,1.3201488e7,5.0,1.02091712e8,3.0937e6,4.3602e6,1.6993836e7,5.0,1.15678597e8,3.8560e6,3.7344e6,1.4775717e7,495970.0,0,…,65,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,6432,65044,0,20,9.1184e6,6.3678e6,1.982748e7,1.657073e6,7.8105e6,3.2299e6,1.3201488e7,5.202422e6,"""suspicious""","""data/cicflowmeter/v1.1/year=20…",38,"""ptmpla""",-0.389427,3,null


In [10]:
mawilab_sample_2011_n10_000.select(target_col).unique()

Label
str
"""notice"""
"""anomalous"""
"""benign"""
"""suspicious"""


In [11]:
preprocessor = CicFlowMeterPreprocessor()

In [12]:
preprocessor.fit(mawilab_sample_2011_n10_000)

In [13]:
mawilab_sample_2011_n10_000_transformed = preprocessor.transform(mawilab_sample_2011_n10_000)
mawilab_sample_2011_n10_000_transformed.head()

Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,…,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol_0,Protocol_17,Protocol_6,Label
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u8,u8,u8,bool
0.024582,0.00017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.6134e-7,0.024582,0.0,0.024582,0.024582,0.024582,0.024582,0.0,0.024582,0.024582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000298,0.0,6.6967e-7,0.0,…,0.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000693,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,false
0.939388,0.001192,0.003926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0511e-7,0.039141,0.048932,0.121012,0.000008,0.895435,0.12792,0.153735,0.325295,0.018153,0.938577,0.059078,0.071523,0.174397,0.000008,0.0,0.0,0.0,0.0,0.000915,0.004203,6.2634e-8,0.000002,…,0.0,0.0,0.00318,0.004017,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.994553,0.997818,0.0,0.357143,0.045187,0.030557,0.06908,0.026231,0.076546,0.039115,0.121012,0.045585,0,0,1,true
0.545805,0.088355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.001052,0.003762,0.01231,0.0,0.545805,0.001052,0.003762,0.01231,8.3334e-9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022128,0.0,0.000008,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18018,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,true
0.049852,0.006469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000007,0.001312,0.004342,0.014438,0.0,0.049852,0.001312,0.004342,0.014438,8.3334e-9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004149,0.0,0.000007,0.0,…,0.0,0.0,0.00159,0.006267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006584,0.0,0.0,0.0,0.001648,0.0,0.0,0.357143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,true
0.981395,0.005618,0.007159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.3527e-7,0.015334,0.029941,0.110013,3.3333e-8,0.850766,0.025781,0.051555,0.141616,4.1667e-8,0.964133,0.032366,0.04427,0.123844,0.004163,0.0,0.0,0.0,0.0,0.003617,0.007159,2.8037e-7,0.000003,…,0.0,0.0,0.00318,0.010445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098146,0.992508,0.0,0.357143,0.083309,0.098743,0.18115,0.01514,0.065088,0.044468,0.110013,0.043354,0,0,1,true


In [14]:
# constant columns
mawilab_sample_2011_n10_000_transformed.select([
    pl.col(col).n_unique() for col in mawilab_sample_2011_n10_000_transformed.columns
    if mawilab_sample_2011_n10_000_transformed.select(pl.col(col).n_unique()).item() < 2
])


Fwd Packet Length Std,Bwd Packet Length Std,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Packet Length Std,Packet Length Variance,URG Flag Count,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1,1,1,1,1,1,1,1,1,1,1
